In [8]:
import pandas as pd
import os
import json

In [14]:
for root, dirs, files in os.walk('../data/datasets/ocr/converted'):
    for file in files:
        if file.endswith(".jsonl"):
            
            input_file = os.path.join(root, file)
            print(input_file)
            if 'nzz' in input_file:
                dataset = pd.read_json(input_file, lines=True)
                print(dataset.head())
                with open(input_file) as f:
                    lines = f.read().splitlines()

../data/datasets/ocr/converted/.jsonl
../data/datasets/ocr/converted/overproof.jsonl


In [13]:
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']
df_inter['json_element'].apply(json.loads)
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final

,filename,ocr.line,ocr.sentence,ocr.region,groundtruth.line,groundtruth.sentence,groundtruth.region
0,../../data/datasets/ocr/original/impresso-nzz/...,Züricher Zeitung. |,Züricher Zeitung. |,Züricher Zeitung. |,Züricher,Züricher,Züricher Zeitung.
1,../../data/datasets/ocr/original/impresso-nzz/...,Freknkreich; (Unterwerfung einiger Chouans-Che...,Freknkreich; (Unterwerfung einiger Chouans-Che...,Freknkreich; (Unterwerfung einiger Chouans-Che...,Frankreich;,Frankreich; burg.,Frankreich; (Unterwerfung einiger Chouans-Chef...
2,../../data/datasets/ocr/original/impresso-nzz/...,"Frankreich. Paris, 7 Pluv. (17 Jan.)","Frankreich. Paris, 7 Pluv. (17 Jan.) Das Tribu...","Frankreich. Paris, 7 Pluv. (17 Jan.) Das Tribu...",Frankreich.,Frankreich. Das Dupuy nem nannt. mand unter re...,"Frankreich. Paris, 7 Pluv. (27 Jan.) Das Tribu..."
3,../../data/datasets/ocr/original/impresso-nzz/...,"General Hedouville meldet aus Angers, daß auch","General Hedouville meldet aus Angers, daß auch...","General Hedouville meldet aus Angers, daß auch...",General,"General die hat, NiederLoire, Vilaine, gekehrt...","General Hedouville meldet aus Angers, daß auch..."
4,../../data/datasets/ocr/original/impresso-nzz/...,"fen Tcustaint, welchen der Chouanö-Chef Charles","fen Tcustaint, welchen der Chouanö-Chef Charle...","fen Tcustaint, welchen der Chouanö-Chef Charle...",fen,"fen von fen, Kriegsgerichts man klamationen","fen Toustaint, welchen der Chouans-Chef Charle..."
...,...,...,...,...,...,...,...
560,../../data/datasets/ocr/original/impresso-nzz/...,"Cadix, vom 10. Junii Ein Chebek bringt","Cadix, vom 10. Junii Ein Chebek bringt uns dir...","Cadix, vom 10. Junii Ein Chebek bringt uns dir...","Cadix,","Cadix, uns Convoy laufen lich seyen. Cantin de...","Cadix, vom 10. Junii. Ein Chebek bringt uns di..."
561,../../data/datasets/ocr/original/impresso-nzz/...,"Muros, vom yten Iumi. Die königliche","Muros, vom yten Iumi. Die königliche Courrier ...","Muros, vom yten Iumi. Die königliche Courrier ...","Muros,","Muros, Courrier-Fregatte D. Aprill canischen","Muros, vom 9ten Junii. Die königliche Courrier..."
562,../../data/datasets/ocr/original/impresso-nzz/...,gestern hier eingelaufen. Ohngefähr 20. Meilen,gestern hier eingelaufen. Ohngefähr 20. Meilen...,gestern hier eingelaufen. Ohngefähr 20. Meilen...,gestern,"gestern vom andere, schon Der und seinem wegen...",gestern hier eingelaufen. Ohngefähr 20. Meilen...
563,../../data/datasets/ocr/original/impresso-nzz/...,Frankreich.,Frankreich.,Frankreich.,Frankreich.,Frankreich.,Frankreich.


In [5]:
def unpack(x):
    rv = []
    for v in x:
        if isinstance(v, dict):
            rv.append([*v.values()][0])
        else:
            rv.append(v)
    return rv

dataset = dataset.apply(unpack)
print(dataset.head())

                                            filename  \
0  ../../data/datasets/ocr/original/impresso-nzz/...   
1  ../../data/datasets/ocr/original/impresso-nzz/...   
2  ../../data/datasets/ocr/original/impresso-nzz/...   
3  ../../data/datasets/ocr/original/impresso-nzz/...   
4  ../../data/datasets/ocr/original/impresso-nzz/...   

                                                 ocr  groundtruth  
0                                Züricher Zeitung. |     Züricher  
1  Freknkreich; (Unterwerfung einiger Chouans-Che...  Frankreich;  
2               Frankreich. Paris, 7 Pluv. (17 Jan.)  Frankreich.  
3     General Hedouville meldet aus Angers, daß auch      General  
4    fen Tcustaint, welchen der Chouanö-Chef Charles          fen  


In [6]:
dataset.columns

Index(['filename', 'ocr', 'groundtruth'], dtype='object')